<a href="https://colab.research.google.com/github/Kaptsha/test-IA/blob/main/Test_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-lnpkvdg2
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-lnpkvdg2
  Resolved https://github.com/unslothai/unsloth.git to commit 1ef71e8b9c570d59470ba5a40ed49064db295f43
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.1.7-py3-none-any.whl size=174898 sha256=cb19f184c3687cc8af75834554f90340937e5810cad02b610021ace5478d8754
  Stored in directory: /tmp/pip-ephem-wheel-cache-p6esbzlp/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.1.7
    Uninstalling unsloth-2025.1.7:
      Successfully uninstalled unsloth-2025.1.7


In [ ]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from google.colab import drive

# Se connecter à Google Drive
drive.mount('/content/drive')

# Load dataset from text file in Google Drive
def load_custom_dataset(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        texts = file.readlines()
    return [{"text": text.strip()} for text in texts]

# Model and training configuration
max_seq_length = 2048
dtype = None # Auto-detect
load_in_4bit = True

# Load pre-trained Phi-4 model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/phi-4",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

# Prepare model for fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank of LoRA adapter
    target_modules = ["q_proj", "k_proj", "v_proj"],
    lora_alpha = 16,
    lora_dropout = 0 # Recommended 0
)

# Load your dataset from Google Drive
# Remplacez " chemin/vers/votre/fichier.txt" par le chemin d'accès à votre fichier dans Google Drive
dataset = load_custom_dataset('/content/drive/MyDrive/tmp/piege-donjon.txt')


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
==((====))==  Unsloth 2025.1.7: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from unsloth import is_bfloat16_supported
from transformers import AutoTokenizer

# Assuming you have your tokenizer loaded or defined earlier as 'tokenizer'
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=max_seq_length)

# Apply tokenization to your dataset
tokenized_datasets = [tokenize_function(example) for example in dataset]

print (tokenized_datasets)

training_args = TrainingArguments(
    output_dir="./results",
    report_to="none",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
)

# Create DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) # mlm=False for causal language modeling

# Training arguments
trainer = Trainer(
    model=model,
    train_dataset=tokenized_datasets,
    args=training_args, # Pass the TrainingArguments object
    data_collator=data_collator # Add the data collator
)

# Start fine-tuning
trainer.train()

model = model.merge_and_unload()

# Save the fine-tuned model
model.save_pretrained("fine_tuned_phi4")
tokenizer.save_pretrained("fine_tuned_phi4")

In [ ]:
!pip install llama-cpp-python

In [ ]:
!pip install transformers torch pandas sentencepiece


In [ ]:
from transformers import AutoConfig, AutoModelForCausalLM
import llama_cpp


# Load the configuration from the fine-tuned model directory
config = AutoConfig.from_pretrained("./fine_tuned_phi4")  # Assuming you saved as "fine_tuned_phi4"

# Update the config with the model type
config.model_type = "gpt2" # Phi-4 is based on GPT2

# Load the model using the updated configuration
# Changed from './results' to './fine_tuned_phi4'
model = AutoModelForCausalLM.from_pretrained(
    "./fine_tuned_phi4",  # Load from where you saved it
    config=config,  # Provide the updated config
    device_map="auto",
    torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained("fine_tuned_phi4") # Load from where you saved it

# Exporter en GGUF
llama_cpp.convert.convert_model(
    "./fine_tuned_phi4", # Changed from './results' to './fine_tuned_phi4'
    "./results GGUF",
    model_type="llama",
    quantize="llama",
    vocab_only=True,
    keep_checkpoints=False,
)